<a href="https://colab.research.google.com/github/ioradlin/NAT_project/blob/Encoder/NAT_Encoder_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def to_txt(x, name):
  with open(name, 'w') as file:
      file.write(x)

def read_txt(name):
  with open(name, 'r') as file:
      temp= file.read()
  return temp

## Transformer model

based on : https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [4]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

#### Encoder

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

#### Decoder

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):

      # src mask
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
      # tgt mask
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        #output= nn.functional.softmax(output, dim=2)

        return output

In [9]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, 30))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, 40))  # (batch_size, seq_length)

In [10]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(1):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.668708801269531


## NAT ENcoder

In [40]:
import torch
import torch.nn as nn

class NATEncoder(nn.Module):
    def __init__(self, src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(NATEncoder, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fertility_fc = nn.Linear(d_model, 50) #L=50 from NAT paper
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src):

        # src mask
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        return src_mask

    def forward(self, src):
        src_mask = self.generate_mask(src)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        #print(enc_output.shape)
        output = self.fertility_fc(enc_output)
        return output

In [41]:
#Encoder (fertility predictor)
src_vocab_size = 1000; d_model = 512; num_heads = 8; num_layers = 6
d_ff = 2048; max_seq_length = 100; dropout = 0.1

encoder = NATEncoder(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

## Encoder용 data preprocess

#### Load datasets

#### Load corpus, vocab, fertility label

In [13]:
import pickle

with open('train_ft_bpe.pkl', 'rb') as file:
     ft = pickle.load(file)

with open('src_vocab.pkl', 'rb') as file:
     src_vocab = pickle.load(file)

with open('tgt_vocab.pkl','rb') as file:
     tgt_vocab = pickle.load(file)

In [14]:
## BPE tokenized

bpe_de= read_txt('bpe_de.txt')
bpe_en= read_txt('bpe_en.txt')

#### **dataset** preprocess

In [15]:
de= bpe_de.split('\n')
en= bpe_en.split('\n')

In [16]:
from collections import Counter
lengths= [len(d) for d in de]

i = 0
for length in lengths:
  if length<=512:
    i+=1

i/len(lengths)

## 99% 이상이 512 이상 길이이므로, 512를 max_len으로 사용

0.9959310779523087


id로 전환, padding & truncation으로 max_len 통일

In [17]:
def tokens_to_ids(text, vocab, max_seq_len=512):
    preprocessed = []

    for line in text:
        tokens = line.split()
        ids = [vocab.get(token, vocab.get('[UNK]')) for token in tokens]

        # padding & trunc는 collate_fn에서 처리! (batch별로 local max_len 써야 효율적)

        # if len(ids) > max_seq_len:
        #     ids = ids[:max_seq_len]
        # else:
        #     ids.extend([vocab['[PAD]']] * (max_seq_len - len(ids)))

        preprocessed.append(ids)

    return preprocessed

In [18]:
tokened_de= tokens_to_ids(de, src_vocab)
tokened_en= tokens_to_ids(en, tgt_vocab)

In [19]:
i=0
for d, f in zip(tokened_de, ft):
  if len(d)!=len(f):
    i+=1

i #엥 ??.. 1%니까 일단.. 빼고 할까요..

1985

In [20]:
data=[]
for d, e, f in zip(tokened_de, tokened_en, ft):
  if len(d)==len(f):
    data.append({'de':tokened_de[i],'en':tokened_en[i],'ft':ft[i]})

In [21]:
len(data)

158254

#### ✰ Loader & collate_fn ✰

In [22]:
# PAD token이 원래 0인데, CE 계산할때 ignore = 0 해주면 fertility 예측값인 0도 무시되서
# PAD token의 인덱스를 새로운 값 10135로 바꿔야할거같음 (tgt_vocab은 이보다 짧음)

pad_id= len(src_vocab)

In [35]:
import torch
from torch.nn.utils.rnn import pad_sequence

def tensor_pad_collate_fn(batch, pad_id=pad_id, max_sequence_length=512):
    padded_data = {}

    # Calculate max_len for each key separately
    max_lens = {key: min(max(len(item[key]) for item in batch if key in item), max_sequence_length) for key in batch[0].keys()}

    for key in batch[0].keys():
        if key in max_lens:
            max_len = max_lens[key]
            sequences = [torch.tensor(item[key][:max_len] if len(item[key]) > max_len else item[key] + [pad_id] * (max_len - len(item[key])), dtype=torch.long) for item in batch if key in item]
            padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=pad_id)
            padded_data[key] = padded_sequences

    return padded_data


In [38]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

In [47]:
## shape check ! (de와 ft는 같아야함) // 2천개 행 제거 후.. 진정됨

for b in train_loader:
  if b['de'].shape!=b['ft'].shape:
    print('??', b['de'].shape, b['ft'].shape)
    print(b['de'], b['ft'])

## Encoder Training (with gpu)

In [48]:
#TransformerEncoder init & to cuda

src_vocab_size = len(src_vocab); d_model = 512; num_heads = 8; num_layers = 6
d_ff = 2048; max_seq_length = 512; dropout = 0.1

encoder = NATEncoder(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)

print(device)

cuda


In [52]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # pad_id: special token index / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
optimizer = optim.Adam(encoder.parameters(), lr=0.01, betas=(0.9, 0.98), eps=1e-9)
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

encoder.train()

for epoch in range(20):

    for i, batch in enumerate(train_loader):

        src = batch['de'].to(device)
        ft_label = batch['ft'].to(device)

        optimizer.zero_grad()

        output = encoder(src)

        # L= 50 (fertility 범위: 0~49) from paper
        loss = criterion(output.contiguous().view(-1,50), ft_label.contiguous().view(-1))

        loss.backward()

        optimizer.step()


    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.9537234306335449
Epoch: 2, Loss: 0.9549230337142944
Epoch: 3, Loss: 0.9538436532020569
Epoch: 4, Loss: 0.9561002850532532
Epoch: 5, Loss: 0.9565297961235046
Epoch: 6, Loss: 0.9556589722633362
Epoch: 7, Loss: 0.9561173319816589
Epoch: 8, Loss: 0.9535827040672302
Epoch: 9, Loss: 0.9557235836982727
Epoch: 10, Loss: 0.9559870362281799
Epoch: 11, Loss: 0.9549953937530518
Epoch: 12, Loss: 0.9558344483375549
Epoch: 13, Loss: 0.9560945630073547
Epoch: 14, Loss: 0.9572625160217285
Epoch: 15, Loss: 0.954663872718811
Epoch: 16, Loss: 0.9561998844146729
Epoch: 17, Loss: 0.9555560350418091
Epoch: 18, Loss: 0.9587217569351196
Epoch: 19, Loss: 0.9543480277061462
Epoch: 20, Loss: 0.95745450258255


In [53]:
torch.save(encoder.state_dict(), 'NATencoder_epoch20')

In [71]:
encoder2 = NATEncoder(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
encoder2.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # pad_id: special token index / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
optimizer = optim.Adam(encoder2.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, collate_fn= tensor_pad_collate_fn)

In [72]:
encoder2.train()

for epoch in range(1):

    for i, batch in enumerate(train_loader):

        src = batch['de'].to(device)
        ft_label = batch['ft'].to(device)

        optimizer.zero_grad()

        output = encoder2(src)

        # L= 50 (fertility 범위: 0~49) from paper
        loss = criterion(output.contiguous().view(-1,50), ft_label.contiguous().view(-1))

        loss.backward()

        if i%1000==0:
          print(f"done: {i*batch_size}, Loss: {loss.item()}")

        optimizer.step()



    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

done: 0, Loss: 4.236850261688232
done: 32000, Loss: 5.042446105107956e-07
done: 64000, Loss: 5.854027040896881e-09
done: 96000, Loss: 1.0909777969914103e-08
done: 128000, Loss: 1.8626450382086546e-09
Epoch: 1, Loss: 1.1556001489054779e-08


lr 낮추니까 CE loss는 확 줄어드는데, 여전히 1 Epoch 안되서 빠르게 수렴해버림

(한 Epoch을 다 돌지 않았을 때는 Epoch내 데이터의 loss는 validation loss나 마찬가진데도)

### Example (참고용, transformer)

In [ ]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 5
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (16, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (16, max_seq_length))  # (batch_size, seq_length)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0) #0: special token / softmax 계산을 내장하므로 unnormalized logits(scroes) 그대로 씀..!!
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data)

    loss = criterion(output.contiguous().view(-1,tgt_vocab_size), tgt_data.contiguous().view(-1))

    # print(output.contiguous().view(-1,tgt_vocab_size).shape, tgt_data.contiguous().view(-1).shape)
    # print('loss',loss)
    loss.backward()
    optimizer.step()
    # break
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")